In [10]:
#block 1st

#import Predction Models with relevant objects
%run ./Step3_Classifier_V10.ipynb

In [11]:
#block 2nd
# predict_model1_cb is version of predict predict_model1 with combinatons of max_stems in doc
# Predict_Model1 based on maximum number of categories of max_stems in doc

def predict_model1_cb(doc,s,train_X,train_y):
    #analyze doc with s combinations of max_stems in doc
    outcome=analyze_doc_s(doc,train_X,train_y,s)
    #check whether outcome is empty, or not
    if len(outcome)==0:
        return "No Prediction"
    
    
    #number of categories of max_stems in doc
    category_counts=[0,0,0,0]
    #iterator for categories
    for i in range(0,4):
            #iterator for max_stems
            for j in range(0,len(outcome)):
                #counting for categories for each_max_stem
                    if outcome[j][4]==main_categories[i]:
                        category_counts[i]=category_counts[i]+1
                        
    #all categories of max_stems of doc is same, prediction is that category
    if sum(category_counts)==max(category_counts):      
            return main_categories[category_counts.index(max(category_counts))]
    #if there is just two labels, then predicition is label having second largest count
    else:
        temp=category_counts.copy()
        #set original max to 0, then second max can be obtained by max function
        temp[temp.index(max(temp))]=0
        return main_categories[temp.index(max(temp))]

#agent method
#for  X_test.apply() function n=2
def agent_model1_cb(doc):
    return predict_model1_cb(doc,X_train,y_train)









In [12]:
#block 3rd
# predict_model2_cb is version of predict predict_model2 with combinatons of max_stems in doc
#model2 based on count of categories weighted by total of counts (of categories) in train set,
#thereby effect of unbalanced categories is expected to be absorbed
def predict_model2_cb(doc,s,train_X,train_y):
    #analyze doc with n combinations of max_stems in doc
    outcome=analyze_doc_s(doc,train_X,train_y,s)
    #check whether outcome is empty, or not
    if len(outcome)==0:
        return "No Prediction"
    
    
    
    #sum of counts for each categories
    sum_category_counts=[0,0,0,0]
    #count of categories weighted by total of counts (for each categories) in train set,
    sum_category_probs=[0,0,0,0]
    #iterator for categories
    for i in range(0,4):
        #iterator for max_stems of doc
        for j in range(0,len(outcome)):
            #sum for counts of each max_stem categories
            sum_category_counts[i]=sum_category_counts[i]+outcome[j][2][i]
        # get sums weighted by counts (of categories) in train set   
        sum_category_probs[i]=round(sum_category_counts[i]/train_counts[i],4)
    
    return main_categories[sum_category_probs.index(max(sum_category_probs))]

#agent method
#for  X_test.apply() function n=2
def agent_model2_cb(doc):
    return predict_model2_cb(doc,2,X_train,y_train)

In [13]:
#block 4th
# predict_model3_cb is version of predict predict_model3 with combinatons of max_stems in doc
#model3 based  mean of non-zero probablities of categories 
def predict_model3_cb(doc,s,train_X,train_y):
    #analyze doc with s combinations of max_stems in doc
    outcome=analyze_doc_s(doc,train_X,train_y,s)
    #check whether outcome is empty, or not
    if len(outcome)==0:
        return "No Prediction"
    
    #list  non-zero probabilities for each categories
    non_zero_probs=[[],[],[],[]]
    # means of non-zero probabilities for each categories
    nz_probs_means=[0]*4
    #iterator for categories
    for i in range(0,4):
        #iterator for max_stems of doc
        for j in range(0,len(outcome)):
            #append non-zero probablities of max_stem to list of each categories
            if outcome[j][3][i] >0:
                non_zero_probs[i].append(outcome[j][3][i])
        #control case divide-by-zero error in case of empty list        
        if len(non_zero_probs[i]) !=0:
            #set mean of probabilities for non-zero counts over categories
            nz_probs_means[i]=round(sum(non_zero_probs[i])/len(non_zero_probs[i]),4)    
            
    # get category with maximum of means of probabilities         
    return main_categories[nz_probs_means.index(max(nz_probs_means))]

#agent method
#for  X_test.apply() function n=2
def agent_model3_cb(doc):
    return predict_model3_cb(doc,2,X_train,y_train)

In [14]:
#block 5th
# predict_model4_cb is version of predict predict_model4 with combinatons of max_stems in doc
#model4 based  max of probablities of categories 
def predict_model4_cb(doc,s,train_X,train_y):
    #analyze doc with s combinations of max_stems in doc
    outcome=analyze_doc_s(doc,train_X,train_y,s)
    #check whether outcome is empty, or not
    if len(outcome)==0:
        return "No Prediction"
    
    
    #list of probabilities for each categories
    probs=[[],[],[],[]]
    # maximum of probabilities for each categories
    max_prob_list=[0]*4
    #iterator for categories
    for i in range(0,4):
        #iterator for max_stems of doc
        for j in range(0,len(outcome)):
            #append probabilities of max_stems in doc
            probs[i].append(outcome[j][3][i])
        
        #set maximum probabilities of max_stems for each categories
        max_prob_list[i]=max(probs[i])   
            
    # get category with maximum probabilities in all max_stems in doc
    return main_categories[max_prob_list.index(max(max_prob_list))]

   

#agent method
#for  X_test.apply() function n=2
def agent_model4_cb(doc):
    return predict_model4_cb(doc,2,X_train,y_train)

In [15]:
#block 6th
# predict_model5_cb is version of predict predict_model5 with combinatons of max_stems in doc
#model5 based on length of max_stems and count of categories weighted by total of counts (of categories) in train set
#thereby effect of length of max_stems is added to the model, 
#it is assumed that longer word, greater effect on finding category
#On the other hand with count of categories weighted by total of counts (of categories) in train set,
#unbalanced case in data-set is not neglected

def predict_model5_cb(doc,s,train_X,train_y):
    #analyze doc with s combinations of max_stems in doc
    outcome=analyze_doc_s(doc,train_X,train_y,s)
    #check whether outcome is empty, or not
    if len(outcome)==0:
        return "No Prediction"
    #score (in direct proportion to both length and counts of max_stems) list 
    scores=[[],[],[],[]]
    #list of maximum of score for each categories
    max_score_list=[0]*4
    #iterator for categories
    for i in range(0,4):
        #iterator for max_stems of doc
        for j in range(0,len(outcome)):
            #value1, length of max_stems
            value1=outcome[j][1]
            #value2, count of categories weighted by total of counts (of categories) in train set
            value2=outcome[j][2][i]/train_counts[i]
            #value3, a score in direct proportion to both length and counts of max_stems
            value3=value1*value2
            #append rounded value3
            scores[i].append(round(value3,4))
        
        #set maximum score for each categories   
        max_score_list[i]=max(scores[i])
    
      
    # get category with maximum scores in all max_stems in doc
    return main_categories[max_score_list.index(max(max_score_list))]

#agent method
#for  X_test.apply() function n=2
def agent_model5_cb(doc):
    return predict_model5_cb(doc,2,X_train,y_train)

In [16]:
#block 7th
#Integrated_model of 1-5 predict model with combinations
def Integrated_model_cb(doc,s,train_X,train_y):
    #analyze doc with s combinations of max_stems in doc
    outcome=analyze_doc_s(doc,train_X,train_y,s)
    #check whether outcome is empty, or not
    if len(outcome)==0:
        return ["No Prediction"]*5
    
    #(model 1)
    category_counts=[0,0,0,0]
    
    #(model 2)
    sum_category_counts=[0,0,0,0]
    sum_category_probs=[0,0,0,0]
    
     #(model 3)
    non_zero_probs=[[],[],[],[]]
    nz_probs_means=[0]*4
    
     #(model 4)
    probs=[[],[],[],[]]
    max_prob_list=[0]*4
    
    #(model 5)
    scores=[[],[],[],[]]
    max_score_list=[0]*4
    
    
    
    for i in range(0,4):
            for j in range(0,len(outcome)):    
                
                 #(model 1)
                if outcome[j][4]==main_categories[i]:
                    category_counts[i]=category_counts[i]+1
                        
                #(model 2)
                sum_category_counts[i]=sum_category_counts[i]+outcome[j][2][i]
                
                #(model 3)
                if outcome[j][3][i] >0:
                    non_zero_probs[i].append(outcome[j][3][i])
                
                #(model 4)
                probs[i].append(outcome[j][3][i])
                
                 #(model 5)
                value1=outcome[j][1]            
                value2=outcome[j][2][i]/train_counts[i]
                value3=value1*value2           
                scores[i].append(round(value3,4))
                
            #(model 2)
            sum_category_probs[i]=round(sum_category_counts[i]/train_counts[i],4)
            
            #(model 3)
            if len(non_zero_probs[i]) !=0:
                nz_probs_means[i]=round(sum(non_zero_probs[i])/len(non_zero_probs[i]),4)  
                
            #(model 4)
            max_prob_list[i]=max(probs[i])
            
            #(model 5)
            max_score_list[i]=max(scores[i])
            
            
    #(model 1)
    if sum(category_counts)==max(category_counts):
        Predict1=main_categories[category_counts.index(max(category_counts))]
    else:
        temp=category_counts.copy()
        temp[temp.index(max(temp))]=0
        Predict1=main_categories[temp.index(max(temp))]
        
    #(model 2)
    Predict2=main_categories[sum_category_probs.index(max(sum_category_probs))]
    
    #(model 3)
    Predict3=main_categories[nz_probs_means.index(max(nz_probs_means))]
    
    #(model 4)
    Predict4=main_categories[max_prob_list.index(max(max_prob_list))]   
    
    #(model 5)
    Predict5=main_categories[max_score_list.index(max(max_score_list))]
    
    
    
    
    return [Predict1,Predict2,Predict3,Predict4,Predict5]

   

#agent method
#for  X_test.apply() function n=2
def agent_model_integrated_cb(doc):
    return Integrated_model_cb(doc,2,X_train,y_train)

In [1]:
#block 8th
#test for predicts models_cb 1-5 and Integrated model_cb 
#import time
#import random
#n=random.randint(1,train_count)
#print(n)
#doc=X_train[X_train.index[n]]

#start time
#start_time=time.time()
#print(predict_model1_cb(doc,2,X_train,y_train))
#print(predict_model2_cb(doc,2,X_train,y_train))
#print(predict_model3_cb(doc,2,X_train,y_train))
#print(predict_model4_cb(doc,2,X_train,y_train))
#print(predict_model5_cb(doc,2,X_train,y_train))

#show run-time
#print("run time for models runs-for-seperately (sec)")
#print(time.time()-start_time)

#start time
#start_time=time.time()
#print(Integrated_model_cb(doc,2,X_train,y_train))

#show run-time
#print("run time for Integrated_model (sec) ")
#print(time.time()-start_time)


